In [0]:
%pip install nameparser

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from nameparser import HumanName # Will be installed via pipeline libraries

import re
import unicodedata
from functools import reduce
import pandas as pd

# ---------------------------------------------------------------------------
# Configuration & Constants
# ---------------------------------------------------------------------------
# Final output table name, equivalent to current 'openalex.works.sources_combined'
# or your chosen name for the parallel run (e.g., 'locations_streamed')
FINAL_OUTPUT_TABLE_NAME = "locations_parsed"

UPSTREAM_SOURCES = {
    "crossref": "openalex.crossref.crossref_works",
    "datacite": "openalex.datacite.datacite_works",
    "pdf": "openalex.pdf.pdf_works",
    "pubmed": "openalex.pubmed.pubmed_works",
    "repo": "openalex.repo.repo_works",
    "repo_backfill": "openalex.repo.repo_works_backfill",
    "mag": "openalex.mag.mag_works",
    "landing_page": "openalex.landing_page.landing_page_works"
}

MERGE_COLUMN_NAME = "merge_key"

# Walden Schema definition (as per your Locations.py)
# This is the target schema for the first major normalization step.
# It does NOT initially contain authors_exist or authors.author_key as per your notebook.
# These will be added in subsequent transformations.
walden_schema = StructType([
    StructField("provenance", StringType(), True), StructField("native_id", StringType(), True),
    StructField("native_id_namespace", StringType(), True), StructField("title", StringType(), True),
    StructField("normalized_title", StringType(), True),
    StructField("authors", ArrayType(StructType([
        StructField("given", StringType(), True), StructField("family", StringType(), True),
        StructField("name", StringType(), True), StructField("orcid", StringType(), True),
        StructField("affiliations", ArrayType(StructType([
            StructField("name", StringType(), True), StructField("department", StringType(), True),
            StructField("ror_id", StringType(), True)])), True),
        StructField("is_corresponding", BooleanType(), True)
    ])), True),
    StructField("ids", ArrayType(StructType([
        StructField("id", StringType(), True), StructField("namespace", StringType(), True),
        StructField("relationship", StringType(), True)])), True),
    StructField("type", StringType(), True), StructField("version", StringType(), True),
    StructField("license", StringType(), True), StructField("language", StringType(), True),
    StructField("published_date", DateType(), True), StructField("created_date", DateType(), True),
    StructField("updated_date", DateType(), True), StructField("issue", StringType(), True),
    StructField("volume", StringType(), True), StructField("first_page", StringType(), True),
    StructField("last_page", StringType(), True), StructField("is_retracted", BooleanType(), True),
    StructField("abstract", StringType(), True), StructField("source_name", StringType(), True),
    StructField("publisher", StringType(), True),
    StructField("funders", ArrayType(StructType([
        StructField("doi", StringType(), True), StructField("ror", StringType(), True),
        StructField("name", StringType(), True), StructField("awards", ArrayType(StringType(), True), True)
    ])), True),
    StructField("references", ArrayType(StructType([
        StructField("doi", StringType(), True), StructField("pmid", StringType(), True),
        StructField("arxiv", StringType(), True), StructField("title", StringType(), True),
        StructField("authors", StringType(), True), StructField("year", StringType(), True),
        StructField("raw", StringType(), True)
    ])), True),
    StructField("urls", ArrayType(StructType([
        StructField("url", StringType(), True), StructField("content_type", StringType(), True)
    ])), True),
    StructField("mesh", StringType(), True), StructField("is_oa", BooleanType(), True)
])


# --- last_name_only UDF and its helpers (VERBATIM) ---
def remove_latin_characters(author):
    if author and any("\u0080" <= c <= "\u02AF" for c in author):
        author = (
            unicodedata.normalize("NFKD", author)
            .encode("ascii", "ignore")
            .decode("ascii")
        )
    return author

def remove_author_prefixes(author):
    if not author: return ""
    prefixes = ["None ", "Array "]
    for prefix in prefixes:
        if author.startswith(prefix):
            author = author.replace(prefix, "")
    return author

def clean_author_name(author_name):
    if not author_name: return ""
    return re.sub(r"[ \-‐.'' ́>]", "", author_name).strip()

def last_name_only(author): 
    if not author:
        return ["", "", ""] 
    author = remove_latin_characters(author)
    author = remove_author_prefixes(author)
    author_name_obj = HumanName(author) 
    first_name = clean_author_name(author_name_obj.first)
    last_name = clean_author_name(author_name_obj.last)
    first_initial = first_name[0] if first_name else ""
    return [ f"{last_name};{first_initial}", f"{first_name}", f"{last_name}" ]

# Schema for the enriched author struct (output of the author processing Pandas UDF)
# This MUST match the structure of the dictionaries returned by the Pandas UDF's internal logic
# AND the walden_schema's authors element type plus author_key.
enriched_author_struct_type = StructType([
    StructField("given", StringType(), True),
    StructField("family", StringType(), True),
    StructField("name", StringType(), True),
    StructField("orcid", StringType(), True),
    StructField("affiliations", walden_schema["authors"].dataType.elementType["affiliations"].dataType, True), 
    StructField("is_corresponding", BooleanType(), True), 
    StructField("author_key", StringType(), True)
])

@F.pandas_udf(ArrayType(enriched_author_struct_type))
def udf_last_name_only(authors_arrays_series: pd.Series) -> pd.Series: # Name matches your original UDF variable
    results = []
    for author_list_for_single_record in authors_arrays_series:
        if author_list_for_single_record is None:
            results.append(None)
            continue
        
        processed_author_list = []
        for author_dict in author_list_for_single_record:
            if author_dict is None:
                processed_author_list.append(None) 
                continue

            name_str_for_parser = author_dict.get("name")
            if not name_str_for_parser:
                given_original = author_dict.get("given", "") or "" 
                family_original = author_dict.get("family", "") or ""
                name_str_for_parser = f"{given_original} {family_original}".strip()
            
            parsed_name_components = ["", "", ""] 
            if name_str_for_parser:
                try:
                    # Calling YOUR original last_name_only Python function
                    parsed_name_components = last_name_only(name_str_for_parser) 
                except Exception: 
                    pass 

            new_given = parsed_name_components[1]
            new_family = parsed_name_components[2]
            reconstructed_name = author_dict.get("name")
            if not reconstructed_name and (new_given or new_family):
                reconstructed_name = f"{new_given or ''} {new_family or ''}".strip()
            
            is_corresponding_val = author_dict.get("is_corresponding")
            is_corresponding_bool = None
            if isinstance(is_corresponding_val, str):
                is_corresponding_bool = is_corresponding_val.lower() == 'true'
            elif isinstance(is_corresponding_val, bool):
                is_corresponding_bool = is_corresponding_val
            
            processed_author_struct = {
                "given": new_given, "family": new_family, "name": reconstructed_name,
                "orcid": author_dict.get("orcid"), 
                "affiliations": author_dict.get("affiliations"), 
                "is_corresponding": is_corresponding_bool,
                "author_key": parsed_name_components[0].lower() if parsed_name_components and parsed_name_components[0] else None
            }
            processed_author_list.append(processed_author_struct)
        results.append(processed_author_list)
    return pd.Series(results)


def f_generate_inverted_index(abstract_string_input): 
    import re 
    import json 
    from collections import OrderedDict 
    
    if not abstract_string_input or not isinstance(abstract_string_input, str): 
        return None
    
    abstract_s = abstract_string_input
    
    # MODIFIED: Combined regex pattern directly in the re.sub call (inline)
    # Replaces newlines, tabs, JATS opening/closing tags, <p>, </p> tags with a single space.
    # The \b replacement was removed as its intent was unclear and potentially problematic.
    abstract_s = re.sub(
        r"\n|\t|<jats:[^>]*?>|</jats:[^>]*?>|<p>|</p>", # Inline regex string
        " ", 
        abstract_s
    )
    
    # Consolidate multiple spaces and strip
    abstract_s = " ".join(abstract_s.split()).strip()

    if not abstract_s: 
        return None

    invertedIndex = OrderedDict()
    words = abstract_s.split()
    for i, word in enumerate(words):
        if word not in invertedIndex: 
            invertedIndex[word] = []
        invertedIndex[word].append(i)
    
    return json.dumps(invertedIndex, ensure_ascii=False) if invertedIndex else None

@F.pandas_udf(StringType())
def udf_f_generate_inverted_index(abstract_series: pd.Series) -> pd.Series: # Name matches your original UDF variable
    # This Pandas UDF calls your 'f_generate_inverted_index' Python function
    return abstract_series.apply(f_generate_inverted_index)

def transform_struct(col_name, source_struct, target_struct):
    target_fields = {f.name: f for f in target_struct.fields}
    source_fields = {f.name: f for f in source_struct.fields}
    expressions = []
    for field_name, field in target_fields.items():
        if field_name in source_fields:
            if isinstance(field.dataType, StructType):
                expressions.append(
                    transform_struct(f"{col_name}.{field_name}", source_fields[field_name].dataType, field.dataType)
                )
            else:
                expressions.append(F.col(f"{col_name}.{field_name}").cast(field.dataType).alias(field_name))
        else:
            expressions.append(F.lit(None).cast(field.dataType).alias(field_name))
    return F.struct(*expressions).alias(col_name)

def transform_array_of_structs(col_name, source_array, target_array):
    target_fields = target_array.elementType.fields
    source_fields = {f.name: f for f in source_array.elementType.fields}
    struct_fields_expr = []
    for field in target_fields:
        if field.name in source_fields:
            if isinstance(field.dataType, StructType):
                nested_expr = transform_struct("x." + field.name, source_fields[field.name].dataType, field.dataType)
                struct_fields_expr.append(f"{nested_expr} AS {field.name}")
            else:
                struct_fields_expr.append(f"x.{field.name} AS {field.name}")
        else:
            struct_fields_expr.append(f"CAST(NULL AS STRING) AS {field.name}")
    struct_expr = f"STRUCT({', '.join(struct_fields_expr)})"
    return F.expr(f"TRANSFORM({col_name}, x -> {struct_expr})").alias(col_name)


def align_column(col_name, source_type, target_type):
    if isinstance(target_type, StructType) and isinstance(source_type, StructType):
        return transform_struct(col_name, source_type, target_type)
    elif isinstance(target_type, ArrayType) and isinstance(source_type, ArrayType):
        if isinstance(target_type.elementType, StructType) and isinstance(source_type.elementType, StructType):
            return transform_array_of_structs(col_name, source_type, target_type)
        else:
            return F.col(col_name).cast(target_type)
    else:
        return F.col(col_name).cast(target_type)

def apply_walden_schema(df, schema):
    schema_fields = {field.name: field for field in schema.fields}
    source_schema_fields = {field.name: field for field in df.schema.fields}
    aligned_columns = []
    for col_name, target_field in schema_fields.items():
        if col_name in source_schema_fields:
            aligned_columns.append(align_column(col_name, source_schema_fields[col_name].dataType, target_field.dataType))
        else:
            aligned_columns.append(F.lit(None).cast(target_field.dataType).alias(col_name))
    return df.select(*aligned_columns)

# --- DOI Normalization (Spark native version for DLT) ---
def normalize_doi_spark_col(doi_string_col_expr):
    return F.regexp_replace(
        F.regexp_extract(
            F.lower(F.trim(F.regexp_replace(doi_string_col_expr, " ", ""))),
            r"(10\.\d+/[^\s]+)", 1),
        r"\u0000", "")

# --- create_merge_column and clean_native_id (VERBATIM from your "Locations Parsed" DLT snippet) ---
def clean_native_id(df, column_name="native_id"):
    return (
            df.withColumn(column_name, F.regexp_replace(F.col(column_name), r"https?://", ""))
            .withColumn(column_name, F.regexp_replace(F.col(column_name), r"/+$", ""))
            .withColumn(column_name, F.regexp_replace(F.col(column_name), r"[^a-zA-Z0-9./:]", ""))
            .withColumn(column_name, F.lower(F.col(column_name))) 
    )

def create_merge_column(df): 
    df_cleaned = clean_native_id(df, "native_id") 
    df_cleaned = df_cleaned.withColumn("title_cleaned_newline", F.regexp_replace(F.col("title"), "\n", " "))
    return df_cleaned.withColumn(MERGE_COLUMN_NAME,
        F.struct(
            F.element_at(F.expr("filter(ids, x -> x.namespace = 'doi' and x.id is not null)"), 1).getField("id").alias("doi"),
            F.element_at(F.expr("filter(ids, x -> x.namespace = 'pmid' and x.id is not null)"), 1).getField("id").alias("pmid"),
            F.element_at(F.expr("filter(ids, x -> x.namespace = 'arxiv' and x.id is not null)"), 1).getField("id").alias("arxiv"),
            F.when(
                (F.expr(f"title_cleaned_newline in (select title from openalex.system.bad_titles)")) |
                (F.length(F.col("title_cleaned_newline")) < 19) |
                (F.col("title_cleaned_newline").isNull()),
                F.concat(F.col("native_id"), F.col("provenance")) 
            ).when(F.col("authors_exist") == False, F.col("normalized_title")
            ).otherwise(F.concat_ws("_", F.col("normalized_title"), F.col("authors").getItem(0).getField("author_key"))
            ).alias("title_author")
        )).drop("title_cleaned_newline")
    
# normalize title and types UDFs

def clean_html(raw_html):
    cleanr = re.compile('<\w+.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def remove_everything_but_alphas(input_string):
    if input_string:
        return "".join(e for e in input_string if e.isalpha())
    return ""

def remove_accents(text):
    normalized = unicodedata.normalize('NFD', text)
    return ''.join(char for char in normalized if unicodedata.category(char) != 'Mn')

def normalize_title(title):
    if not title:
        return ""

    if isinstance(title, bytes):
        title = str(title, 'ascii')

    text = title[0:500]

    text = text.lower()

    # handle unicode characters
    text = remove_accents(text)

    # remove HTML tags
    text = clean_html(text)

    # remove articles and common prepositions
    text = re.sub(r"\b(the|a|an|of|to|in|for|on|by|with|at|from|\n)\b", "", text)

    # remove everything except alphabetic characters
    text = remove_everything_but_alphas(text)

    return text.strip()

def normalize_license(text):
    if not text:
        return None

    normalized_text = text.replace(" ", "").replace("-", "").lower()

    license_lookups = [
        # open Access patterns
        ("infoeureposematicsaccess", "other-oa"),
        ("openaccess", "other-oa"),
        
        # publisher-specific
        ("elsevier.com/openaccess/userlicense", "publisher-specific-oa"),
        ("pubs.acs.org/page/policy/authorchoice_termsofuse.html", "publisher-specific-oa"),
        ("arxiv.orgperpetual", "publisher-specific-oa"),
        ("arxiv.orgnonexclusive", "publisher-specific-oa"),
        
        # creative Commons licenses
        ("ccbyncnd", "cc-by-nc-nd"),
        ("ccbyncsa", "cc-by-nc-sa"),
        ("ccbynd", "cc-by-nd"),
        ("ccbysa", "cc-by-sa"),
        ("ccbync", "cc-by-nc"),
        ("ccby", "cc-by"),
        ("creativecommons.org/licenses/by/", "cc-by"),
        
        # public domain
        ("publicdomain", "public-domain"),
        
        # software/Dataset licenses
        ("mit ", "mit"),
        ("gpl3", "gpl-3"),
        ("gpl2", "gpl-2"),
        ("gpl", "gpl"),
        ("apache2", "apache-2.0")
    ]

    for lookup, license in license_lookups:
        if lookup in normalized_text:
            if license == "public-domain" and "worksnotinthepublicdomain" in normalized_text:
                continue
            return license
    return None

@F.pandas_udf(StringType())
def normalize_license_udf(license_series: pd.Series) -> pd.Series:
    # This Pandas UDF calls your original 'normalize_license' Python function
    return license_series.apply(normalize_license)

@F.pandas_udf(StringType())
def normalize_title_udf(title_series: pd.Series) -> pd.Series:
    # This Pandas UDF calls your original 'normalize_title' Python function
    return title_series.apply(normalize_title)